# LSTM을 이용한 텍스트 생성

## 1. 데이터 전처리

In [46]:
import pandas as pd
import numpy as np
from string import punctuation
from konlpy.tag import Okt
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('C:/Users/Myeong/dding/data/train_10000_mk2.csv')
df.head()

,non_label_sentence,label_sentence,class,binary_class
0,해병대는 자기가 지원해서 가는 거잖아\n응응 시험 같은 것도 있지 않아\n어떻게 해...,1:해병대는 자기가 지원해서 가는 거잖아\n2:응응 시험 같은 것도 있지 않아\n1...,일반 대화,일반 대화
1,오늘 서울은 하루종일 꾸물하다 날씨가\n요즘 날씨가 계속 꾸물하고 비오고\n언제 선...,1:오늘 서울은 하루종일 꾸물하다 날씨가\n2:요즘 날씨가 계속 꾸물하고 비오고\n...,일반 대화,일반 대화
2,남자들은 전립선 비대증이 큰일이에요\n진작에 관리 안 한 걸 후회한다\n치료도 잘 ...,1:남자들은 전립선 비대증이 큰일이에요\n2:진작에 관리 안 한 걸 후회한다\n1:...,일반 대화,일반 대화
3,안녕\n나는 이 날씨에 서핑하러 간다\n와 이제 곧 겨울인데 서핑을 해\n당연히 제...,1:안녕\n1:나는 이 날씨에 서핑하러 간다\n2:와 이제 곧 겨울인데 서핑을 해\...,일반 대화,일반 대화
4,넌 몇살에 결혼 하고 싶어\n35살 난 최대한 즐기고 결혼 할 거야\n그럼 애기는 ...,1:넌 몇살에 결혼 하고 싶어\n2:35살 난 최대한 즐기고 결혼 할 거야\n1:그...,일반 대화,일반 대화


In [47]:
okt = Okt()
def preprocessing(text):
    text = text.strip()
    text = re.sub(r'[^ㄱ-ㅎ가-힣ㅏ-ㅣ0-9]', ' ', text)
    text = text.strip()
    text = ' '.join(okt.morphs(text))
    return text

df['tokenized'] = df['non_label_sentence'].apply(preprocessing)
df

,non_label_sentence,label_sentence,class,binary_class,tokenized
0,해병대는 자기가 지원해서 가는 거잖아\n응응 시험 같은 것도 있지 않아\n어떻게 해...,1:해병대는 자기가 지원해서 가는 거잖아\n2:응응 시험 같은 것도 있지 않아\n1...,일반 대화,일반 대화,해병대 는 자기 가 지원 해서 가는 거 잖아 응응 시험 같은 것 도 있지 않아 어떻...
1,오늘 서울은 하루종일 꾸물하다 날씨가\n요즘 날씨가 계속 꾸물하고 비오고\n언제 선...,1:오늘 서울은 하루종일 꾸물하다 날씨가\n2:요즘 날씨가 계속 꾸물하고 비오고\n...,일반 대화,일반 대화,오늘 서울 은 하루 종일 꾸 물하다 날씨 가 요즘 날씨 가 계속 꾸 물 하고 비오고...
2,남자들은 전립선 비대증이 큰일이에요\n진작에 관리 안 한 걸 후회한다\n치료도 잘 ...,1:남자들은 전립선 비대증이 큰일이에요\n2:진작에 관리 안 한 걸 후회한다\n1:...,일반 대화,일반 대화,남자 들 은 전립선 비대증 이 큰일 이에요 진작 에 관리 안 한 걸 후회 한 다 치...
3,안녕\n나는 이 날씨에 서핑하러 간다\n와 이제 곧 겨울인데 서핑을 해\n당연히 제...,1:안녕\n1:나는 이 날씨에 서핑하러 간다\n2:와 이제 곧 겨울인데 서핑을 해\...,일반 대화,일반 대화,안녕 나 는 이 날씨 에 서핑 하러 간다 와 이제 곧 겨울 인데 서핑 을 해 당연히...
4,넌 몇살에 결혼 하고 싶어\n35살 난 최대한 즐기고 결혼 할 거야\n그럼 애기는 ...,1:넌 몇살에 결혼 하고 싶어\n2:35살 난 최대한 즐기고 결혼 할 거야\n1:그...,일반 대화,일반 대화,넌 몇 살 에 결혼 하고 싶어 35 살 난 최대한 즐기고 결혼 할 거야 그럼 애기 ...
...,...,...,...,...,...
13865,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화,준 하야 넌 대가리 가 왜 이렇게 크냐 내 머리 가 뭐 밥 먹으면 대가리 만 크냐 ...
13866,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화,내 가 지금 너 아들 김길준 데리 고 있어 살리고 싶으면 계좌 에 1억만 보내 예 ...
13867,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화,나 는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살 지 왜 그래 들리겠어 들...
13868,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화,누구 맘대로 여기 서 장사 하래 이 게 무슨 일 입 니까 남 의 구역 에서 장사 하...


In [59]:
tokenized_sen = list(df['tokenized'].sample(1000).values)
tokenized_sen[:3]

['네 회원 님 아 네 입학 문의 는 어디 에 연락 해야 하나 해서요 아 혹시 대표 전화 알려 드려도 될까 요 네 메모 가능하고 불러 주셔도 됩니다 네 혹시 평일 몇 시 까지 문의 받으시나요 평일 은 오전 9시 부터 밤 10시 까지 문의 가 가능하십니다 네 알겠습니다 친절하시네요 네 즐거운 하루 보내세요',
 '야외 활동 도 좋아하나 같이 함 오 산책 매일 하니까 야외 활동 매일 하지 그 뭐 야 원반 던지기 그런 거 할 수 있음 이 산책 이라는 것 이 쉬운 일이 아니라고 진짜 그게 제일 어렵지 원반 은 안 던져 봤는데 공 던지기 는 한다 오 공 잘 주서 오 나 보네 공 잘 줍는다 다시 오지 는 않던데 왜 안 오노 주워 가지 고 와야지 원반 도 한번 사서 던져 볼까 생각 중이 다 근데 아직 아기 라서 입 에 잡 을 수 있으려나 돌아오기는 하는데 나 한테 주지 않더라고 아 왜 그러지 다시 교육 테니스공 같은 건 입 에 물 수 있더라 하하 오 쩌 네 상상 만 해도 귀엽노 너무 귀엽지 가끔 입 찢어질라 해',
 '머리 는 달 고 다니냐 말씀 이 좀 생각 좀 하고 살라 고 틀린 말 했냐 시정 하겠습니다 시정 은 개뿔 대답 안 하냐 죄송합니다 죄송하지 말고 일 을 하라 고 네']

In [60]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_sen)
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

15023


In [61]:
sequences = list()
for sen in tokenized_sen:
    encoded = tokenizer.texts_to_sequences([sen])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]


[[6, 499],
 [6, 499, 84],
 [6, 499, 84, 12],
 [6, 499, 84, 12, 6],
 [6, 499, 84, 12, 6, 6471],
 [6, 499, 84, 12, 6, 6471, 110],
 [6, 499, 84, 12, 6, 6471, 110, 5],
 [6, 499, 84, 12, 6, 6471, 110, 5, 134],
 [6, 499, 84, 12, 6, 6471, 110, 5, 134, 3],
 [6, 499, 84, 12, 6, 6471, 110, 5, 134, 3, 169],
 [6, 499, 84, 12, 6, 6471, 110, 5, 134, 3, 169, 144]]

In [62]:
tokenizer.word_index

{'이': 1,
 '가': 2,
 '에': 3,
 '도': 4,
 '는': 5,
 '네': 6,
 '거': 7,
 '은': 8,
 '나': 9,
 '안': 10,
 '을': 11,
 '아': 12,
 '들': 13,
 '내': 14,
 '너': 15,
 '해': 16,
 '야': 17,
 '다': 18,
 '진짜': 19,
 '요': 20,
 '를': 21,
 '한': 22,
 '것': 23,
 '말': 24,
 '에서': 25,
 '뭐': 26,
 '잘': 27,
 '그': 28,
 '좀': 29,
 '로': 30,
 '못': 31,
 '게': 32,
 '으로': 33,
 '왜': 34,
 '하고': 35,
 '하는': 36,
 '만': 37,
 '너무': 38,
 '나도': 39,
 '그럼': 40,
 '할': 41,
 '수': 42,
 '많이': 43,
 '고': 44,
 '더': 45,
 '우리': 46,
 '때': 47,
 '지금': 48,
 '사람': 49,
 '응': 50,
 '돈': 51,
 '맞아': 52,
 '난': 53,
 '근데': 54,
 '그래': 55,
 '그냥': 56,
 '제': 57,
 '그런': 58,
 '의': 59,
 '아니': 60,
 '생각': 61,
 '저': 62,
 '하하': 63,
 '이야': 64,
 '하면': 65,
 '거야': 66,
 '있어': 67,
 '집': 68,
 '같아': 69,
 '요즘': 70,
 '일': 71,
 '봐': 72,
 '지': 73,
 '랑': 74,
 '어떻게': 75,
 '무슨': 76,
 '알': 77,
 '오늘': 78,
 '까지': 79,
 '해서': 80,
 '오': 81,
 '게임': 82,
 '한테': 83,
 '님': 84,
 '하지': 85,
 '면': 86,
 '그래서': 87,
 '그렇게': 88,
 '와': 89,
 '엄청': 90,
 '그래도': 91,
 '그거': 92,
 '라고': 93,
 '인데': 94,
 '다른': 95,
 '정말': 96,
 '

In [63]:
index_to_word = {value : key for key, value in tokenizer.word_index.items()}
index_to_word

{1: '이',
 2: '가',
 3: '에',
 4: '도',
 5: '는',
 6: '네',
 7: '거',
 8: '은',
 9: '나',
 10: '안',
 11: '을',
 12: '아',
 13: '들',
 14: '내',
 15: '너',
 16: '해',
 17: '야',
 18: '다',
 19: '진짜',
 20: '요',
 21: '를',
 22: '한',
 23: '것',
 24: '말',
 25: '에서',
 26: '뭐',
 27: '잘',
 28: '그',
 29: '좀',
 30: '로',
 31: '못',
 32: '게',
 33: '으로',
 34: '왜',
 35: '하고',
 36: '하는',
 37: '만',
 38: '너무',
 39: '나도',
 40: '그럼',
 41: '할',
 42: '수',
 43: '많이',
 44: '고',
 45: '더',
 46: '우리',
 47: '때',
 48: '지금',
 49: '사람',
 50: '응',
 51: '돈',
 52: '맞아',
 53: '난',
 54: '근데',
 55: '그래',
 56: '그냥',
 57: '제',
 58: '그런',
 59: '의',
 60: '아니',
 61: '생각',
 62: '저',
 63: '하하',
 64: '이야',
 65: '하면',
 66: '거야',
 67: '있어',
 68: '집',
 69: '같아',
 70: '요즘',
 71: '일',
 72: '봐',
 73: '지',
 74: '랑',
 75: '어떻게',
 76: '무슨',
 77: '알',
 78: '오늘',
 79: '까지',
 80: '해서',
 81: '오',
 82: '게임',
 83: '한테',
 84: '님',
 85: '하지',
 86: '면',
 87: '그래서',
 88: '그렇게',
 89: '와',
 90: '엄청',
 91: '그래도',
 92: '그거',
 93: '라고',
 94: '인데',
 95: '다른',
 96: '정말',
 9

In [64]:
max_len = max(len(l) for l in sequences)
max_len

217

In [65]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [66]:
sequences[:3]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [67]:
sequences = np.array(sequences)
x = sequences[:, :-1]
y = sequences[:, -1]
x[:3]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [68]:
y = to_categorical(y, num_classes=vocab_size)

## 모델 설계

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=2)

Epoch 1/200
2915/2915 - 69s - loss: 8.0172 - accuracy: 0.0209 - 69s/epoch - 24ms/step
Epoch 2/200
2915/2915 - 65s - loss: 7.5861 - accuracy: 0.0270 - 65s/epoch - 22ms/step
Epoch 3/200
2915/2915 - 65s - loss: 7.3124 - accuracy: 0.0364 - 65s/epoch - 22ms/step
Epoch 4/200
2915/2915 - 65s - loss: 6.9686 - accuracy: 0.0489 - 65s/epoch - 22ms/step
Epoch 5/200
2915/2915 - 65s - loss: 6.6181 - accuracy: 0.0599 - 65s/epoch - 22ms/step
Epoch 6/200
2915/2915 - 65s - loss: 6.2522 - accuracy: 0.0714 - 65s/epoch - 22ms/step
Epoch 7/200
2915/2915 - 65s - loss: 5.8876 - accuracy: 0.0823 - 65s/epoch - 22ms/step
Epoch 8/200
2915/2915 - 65s - loss: 5.5338 - accuracy: 0.1000 - 65s/epoch - 22ms/step
Epoch 9/200
2915/2915 - 65s - loss: 5.2046 - accuracy: 0.1276 - 65s/epoch - 22ms/step
Epoch 10/200
2915/2915 - 65s - loss: 4.8992 - accuracy: 0.1600 - 65s/epoch - 22ms/step
Epoch 11/200
2915/2915 - 65s - loss: 4.6226 - accuracy: 0.1934 - 65s/epoch - 22ms/step
Epoch 12/200
2915/2915 - 65s - loss: 4.3693 - accura

In [81]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        tokenized = ' '.join(okt.morphs(current_word))
        encoded = tokenizer.texts_to_sequences([tokenized])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            if index == result:
                break
        
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    
    sentence = init_word + sentence
    return sentence

In [82]:
print(sentence_generation(model, tokenizer, '내가', 10))

내가 어려운 부탁 하는 거 알 가요 그러니 든 말씀 할


In [85]:
print(sentence_generation(model, tokenizer, '아니', 10))

아니 지은 씨 내 가 왜 배우 한번 해준 건데 사장


# 문자 단위 RNN(Char RNN)

In [1]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve('http://www.gutenberg.org/files/11/11-0.txt', filename='11-0.txt')

f = open('11-0.txt', 'rb')
sentences = []
for sentence in f:
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.decode('ascii', 'ignore')
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

sentences[:3]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions']

In [2]:
# 문장 토큰화가 의미있게 진행 된 상태가 아님
# 하나의 문자열로 통합
total_data = ' '.join(sentences)
print(len(total_data))

159484


In [3]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print(vocab_size)

56


In [4]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [5]:
index_to_char = {value : key for key, value in char_to_index.items()}

seq_length = 60
n_samples = int(np.floor((len(total_data)-1)/seq_length))
print(n_samples)

2658


In [6]:
train_x = list()
train_y = list()

for i in range(n_samples):
    x_sample = total_data[i * seq_length : (i+1) * seq_length]
    x_encoded = [char_to_index[c] for c in x_sample]
    train_x.append(x_encoded)

    y_sample = total_data[i*seq_length + 1 : (i+1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [7]:
print(train_x[0])
print(train_y[0])
print()
print([index_to_char[i] for i in train_x[0]])
print([index_to_char[i] for i in train_y[0]])

[49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
[37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]

['t', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a']
['h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', '

In [8]:
train_x = to_categorical(train_x)
train_y = to_categorical(train_y)
print(train_x.shape)
print(train_y.shape)

(2658, 60, 56)
(2658, 60, 56)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_x.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 [==============================] - 4s 9ms/step - loss: 3.0705 - accuracy: 0.1822
Epoch 2/80
84/84 [==============================] - 1s 9ms/step - loss: 2.6901 - accuracy: 0.2561
Epoch 3/80
84/84 [==============================] - 1s 9ms/step - loss: 2.3653 - accuracy: 0.3327
Epoch 4/80
84/84 [==============================] - 1s 9ms/step - loss: 2.2231 - accuracy: 0.3691
Epoch 5/80
84/84 [==============================] - 1s 9ms/step - loss: 2.1081 - accuracy: 0.3968
Epoch 6/80
84/84 [==============================] - 1s 9ms/step - loss: 2.0220 - accuracy: 0.4176
Epoch 7/80
84/84 [==============================] - 1s 9ms/step - loss: 1.9485 - accuracy: 0.4371
Epoch 8/80
84/84 [==============================] - 1s 9ms/step - loss: 1.8849 - accuracy: 0.4551
Epoch 9/80
84/84 [==============================] - 1s 9ms/step - loss: 1.8251 - accuracy: 0.4715
Epoch 10/80
84/84 [==============================] - 1s 9ms/step - loss: 1.7743 - accuracy: 0.4840
Epoch 11/80
84/84 [

In [12]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)]
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1], '번 문자', y_char[-1],'로 예측을 시작!')

    x = np.zeros((1, length, vocab_size))

    for i in range(length):
        x[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end='')
        ix = np.argmax(model.predict(x[:, :i+1, :])[0],1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [14]:
result = sentence_generation(model, 100)
print(result)

38 번 문자 i 로 예측을 시작!
1/1 [==============================] - 0s 17ms/step
ing to find her way out. i shall be punished for it now, i sure, she said to herself, and began to ce
